In [19]:
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

# get the current object detection and crop bbox to keep the ration 3:4
def get_bbox_cropp_keep_ration(bbox):
    bbox2 = np.array([0,0,0,0])    

    # need to set the scale to go from Sighthound resolution to original
    x_scale = 4.5
    y_scale = 4.53
    bbox[0] = bbox[0]*x_scale
    bbox[1] = bbox[1]*y_scale
    bbox[2] = bbox[2]*x_scale
    bbox[3] = bbox[3]*y_scale
    width   = bbox[2]-bbox[0]
    x_center= bbox[0]+width/2
    height  = bbox[3]-bbox[1]
    y_center= bbox[1]+height/2
    # normalization for Faster-RCNN
    fix_width = 500
    fix_height = 375
    #print([width,height,bbox])
    #print([x_center,y_center])
    # if the bbox is on the edge of the image
    if x_center < fix_width/2:
        x_center = fix_width/2

    if y_center < fix_height/2:
        y_center = fix_height/2

    if (1920-x_center) < fix_width/2:
        x_center = 1920-fix_width/2


    if (1080-y_center) < fix_height/2:
        y_center = 1080-fix_height/2

    # normalize      
    if width>height:
        expected_height = width*0.75
        expected_width = width
    else:
        expected_width = height/0.75
        expected_height = height
        
    #print([x_center,y_center])
    #print([expected_width,expected_height])
    # extend the bbox to keep the ratio between 500x375
    bbox2[0] = x_center-expected_width/2
    bbox2[1] = y_center-expected_height/2
    bbox2[2] = x_center+expected_width/2
    bbox2[3] = y_center+expected_height/2
    #print(bbox2)
    # increase size of bbox2
    bbox_scale = 0.3
    bbox2[0] = max(0,bbox2[0]-fix_width*bbox_scale)
    bbox2[1] = max(0,bbox2[1]-fix_height*bbox_scale)
    bbox2[2] = min(1920,bbox2[2]+fix_width*bbox_scale)
    bbox2[3] = min(1080,bbox2[3]+fix_height*bbox_scale)
    return bbox2

def get_bbox_exact_cropp(bbox):
    bbox2 = np.array([0,0,0,0])    

    # need to set the scale to go from Sighthound resolution to original
    x_scale = 4.5
    y_scale = 4.53
    bbox[0] = bbox[0]*x_scale
    bbox[1] = bbox[1]*y_scale
    bbox[2] = bbox[2]*x_scale
    bbox[3] = bbox[3]*y_scale
    return bbox


def crop_image(image_path, image_name, bbox, id, cropfile, outpath):

    im_file = os.path.join(image_path, image_name)
    im = Image.open(im_file)  
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(im, aspect='equal')

    bbox2 = get_bbox_exact_cropp(bbox)
    outFile =  outpath  + str(id) + '_' + image_name
    im2 = im.crop((bbox2))
    im2.save(outFile)
    plt.close(fig)
    
    with open(cropfile, 'a') as text_file:
        print("{},{},{},{},{}".format(str(id) + '_' + image_name,bbox2[0],bbox2[1],bbox2[2],bbox2[3]), file=text_file)    

In [21]:
# crop the image based on given BBOX

in_dir = 'z:/OC_RESINET/ResiIndoor/Img_orig/'

annotation_file = 'z:/OC_RESINET/ResiIndoor/CSV/ResiIndoor_annotations_output_all.csv'

out_dir = in_dir + 'Annotation_Crop/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

cropfile = out_dir + 'img_list_bbox.csv'
if os.path.exists(cropfile):
    os.remove(cropfile)


with open(annotation_file, 'r') as file:
    line = file.readline()
    for line in file:
        line = str(line)
        image_name,id,x,y,u,v,s = line.split(',')
        bbox = np.array([float(x),float(y),float(u),float(v)])
        #if image_name[-8:-4] == '1081':
                #print(i)
        crop_image(in_dir, image_name, bbox, id, cropfile, out_dir)

